In [1]:
import pandas as pd
import plotly.express as px
import plotly.io as pio
import plotly.graph_objects as go
pio.templates.default="plotly_white"

In [2]:
data=pd.read_csv('../../../Downloads/rfm_data.csv')

In [3]:
data.head()

,CustomerID,PurchaseDate,TransactionAmount,ProductInformation,OrderID,Location
0,8814,2023-04-11,943.31,Product C,890075,Tokyo
1,2188,2023-04-11,463.70,Product A,176819,London
2,4608,2023-04-11,80.28,Product A,340062,New York
3,2559,2023-04-11,221.29,Product A,239145,London
4,9482,2023-04-11,739.56,Product A,194545,Paris


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   CustomerID          1000 non-null   int64  
 1   PurchaseDate        1000 non-null   object 
 2   TransactionAmount   1000 non-null   float64
 3   ProductInformation  1000 non-null   object 
 4   OrderID             1000 non-null   int64  
 5   Location            1000 non-null   object 
dtypes: float64(1), int64(2), object(3)
memory usage: 47.0+ KB


In [5]:
data.describe()

,CustomerID,TransactionAmount,OrderID
count,1000.000000,1000.00000,1000.000000
mean,5554.789000,513.67781,554071.398000
std,2605.014863,286.09870,264695.448814
min,1011.000000,12.13000,100096.000000
25%,3273.000000,257.12000,313152.000000
50%,5538.000000,523.56500,564671.500000
75%,7821.750000,759.86000,783052.250000
max,9991.000000,999.44000,999695.000000


In [6]:
from datetime import datetime

# Convert 'Purchase date' to datetime
data['PurchaseDate'] = pd.to_datetime(data['PurchaseDate'], errors='coerce')

# Drop rows with invalid date values
data = data.dropna(subset=['PurchaseDate'])

# Calculate recency (days since last purchase)
data['Recency'] = (datetime.now() - data['PurchaseDate']).dt.days

# Calculate frequency of purchase
frequency_data = data.groupby('CustomerID')['OrderID'].count().reset_index()
frequency_data.columns = ['CustomerID', 'Frequency']
data = data.merge(frequency_data, on='CustomerID', how='left')


In [7]:
# Calculate monetary value
monetary_data = data.groupby('CustomerID')['TransactionAmount'].sum().reset_index()
monetary_data.rename(columns={'TransactionAmount': 'MonetaryValue'}, inplace=True)  

# Merge the monetary_data back into the original DataFrame
data = data.merge(monetary_data, on='CustomerID', how='left')


In [8]:
data.head()

,CustomerID,PurchaseDate,TransactionAmount,ProductInformation,OrderID,Location,Recency,Frequency,MonetaryValue
0,8814,2023-04-11,943.31,Product C,890075,Tokyo,185,1,943.31
1,2188,2023-04-11,463.70,Product A,176819,London,185,1,463.70
2,4608,2023-04-11,80.28,Product A,340062,New York,185,1,80.28
3,2559,2023-04-11,221.29,Product A,239145,London,185,1,221.29
4,9482,2023-04-11,739.56,Product A,194545,Paris,185,1,739.56


In [9]:
# Calculate RFM SCORE

In [10]:
# Scoring Criteria for Scoring each RFM value
# Higher scores for lower recency
# Higher scores for higher freq and monerary values
recency_scores=[5,4,3,2,1]
frequency_scores=[1,2,3,4,5]
monetary_score=[1,2,3,4,5]

In [11]:
# RFM
data['RecencyScore']=pd.cut(data['Recency'],bins=5,labels=recency_scores)
data['FrequencyScore']=pd.cut(data['Frequency'],bins=5,labels=frequency_scores)
data['MonetaryScore']=pd.cut(data['MonetaryValue'],bins=5,labels=monetary_score)

In [12]:
data.head()

,CustomerID,PurchaseDate,TransactionAmount,ProductInformation,OrderID,Location,Recency,Frequency,MonetaryValue,RecencyScore,FrequencyScore,MonetaryScore
0,8814,2023-04-11,943.31,Product C,890075,Tokyo,185,1,943.31,1,1,2
1,2188,2023-04-11,463.70,Product A,176819,London,185,1,463.70,1,1,1
2,4608,2023-04-11,80.28,Product A,340062,New York,185,1,80.28,1,1,1
3,2559,2023-04-11,221.29,Product A,239145,London,185,1,221.29,1,1,1
4,9482,2023-04-11,739.56,Product A,194545,Paris,185,1,739.56,1,1,2


In [13]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   CustomerID          1000 non-null   int64         
 1   PurchaseDate        1000 non-null   datetime64[ns]
 2   TransactionAmount   1000 non-null   float64       
 3   ProductInformation  1000 non-null   object        
 4   OrderID             1000 non-null   int64         
 5   Location            1000 non-null   object        
 6   Recency             1000 non-null   int64         
 7   Frequency           1000 non-null   int64         
 8   MonetaryValue       1000 non-null   float64       
 9   RecencyScore        1000 non-null   category      
 10  FrequencyScore      1000 non-null   category      
 11  MonetaryScore       1000 non-null   category      
dtypes: category(3), datetime64[ns](1), float64(2), int64(4), object(2)
memory usage: 74.0+ KB
